# Создание социального графа

В этом блокноте мы выгрузим данные о клубе и построим его социальный граф, который потом визуализируем с помощью JS. 

Предварительно нужно получить доступ к API ВКонтакте: создать токен (схема есть <a href = 'http://all-for-vkontakte.ru/catalog/access-token-vkontakte'>например, здесь</a>).

Создайте себе токен и положите его в файл `secret.json` примерно такого вида:

    {
      "token": "abcdefg123456"
    }
    
    

Важно: в файле должна быть четвёртая, пустая, строка. 

## Технические требования

In [1]:
import json
with open('secret.json', 'r') as f:
    secret = json.load(f)

In [2]:
token = secret['token']
vk_api_version = "5.92"

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import pymorphy2
anl = pymorphy2.MorphAnalyzer();

In [5]:
import warnings

In [6]:
inflect_fixlist = {
    'Айрана': {'gent': 'Айраны', 'ablt': 'Айраной'}
}

In [7]:
def get_inflections(word):
    parsed = anl.parse(word)
    p = None
    for hyp in parsed:
        if hyp.tag.case == 'nomn':
            p = hyp
            break
    if p is None:
        word = word.capitalize()
        if word in inflect_fixlist:
            return inflect_fixlist[word]
        else:
            warnings.warn('Could not parse name "{}", hypotheses are {}'.format(word, parsed))
            return {'gent': word, 'ablt': word}
    result = {}
    for case in ['gent', 'ablt']:
        result[case] = p.inflect({case}).word.capitalize()
    return result
get_inflections('альфия')

{'ablt': 'Альфией', 'gent': 'Альфии'}

# Собственно, задача Каппы Веди

In [8]:
excel = pd.read_excel('Опрос КВ НГ.xlsx')
excel.shape

(85, 23)

In [9]:
excel.head(3).T

,0,1,2
#,70f89da2a65b1222c0bf9c6ad8ada254,83408438908f9bee8e3278fecd369191,a951e6b3e9a78dc7aaf6a977748ac580
Назови своё имя,Тимур,Алина,Арина
и фамилию,Али-Заде,Иванова,Егорова
Член клуба,0,0,1
Из какого ты города?,Москва,Балаково,Москва
Дай ссылку на свою страницу Вконтакте и мы поможем найти твоих друзей на встрече,https://vk.com/timuraz,NaN,https://vk.com/id119458406
Выбери свой первый вуз - бакалавриат,Финансовый университет,NaN,НИУ ВШЭ
Other,NaN,Frankfurt School of Finance & Management,NaN
Укажи свой факультет в бакалавриате,Учет и аудит,International Business Administration,Мировой экономики и мировой политики
Выбери свой первый вуз - магистратура,Финансовый университет,NaN,NaN


In [10]:
PHOTO_COL = 'Загрузи свое фото, на котором четко видно лицо.'
URL_COL = 'Дай ссылку на свою страницу Вконтакте и мы поможем найти твоих друзей на встрече'
CITY_COL = 'Из какого ты города?'
BACHELOR_COL = 'Выбери свой первый вуз - бакалавриат'
OTHER_BACH_COL = 'Other'
JOB_COL = 'Где ты сейчас работаешь?'
HOBBY_COL = 'Какое у тебя хобби?'
INTERESTS_COL = 'Расскажи о своих профессиональных интересах.'
FUN_COL = 'Фраза'
NAME_COL = 'Назови своё имя'
IN_CLUB_COL = 'Член клуба'
PRESENCE_COL = 'Присутствие'

In [11]:
links = excel.loc[:, URL_COL]
links.sample(10)

7        https://vk.com/id166330633
53          https://vk.com/dborunov
33        https://vk.com/tereshanna
17              https://vk.com/feed
11    https://vk.com/svoeobuchnaya 
44                              NaN
61        https://m.vk.com/evgennik
46       https://vk.com/id129531173
68       https://vk.com/eeepavlenko
20         https://vk.com/az_zakria
Name: Дай ссылку на свою страницу Вконтакте и мы поможем найти твоих друзей на встрече, dtype: object

In [12]:
import json
import urllib

In [13]:
def get_user_by_name(username):
    resp = json.loads(
        urllib.request.urlopen(
            'https://api.vk.com/method/users.get?v='+vk_api_version+'&user_ids='+username
            +'&fields=name,photo_50&access_token='+token).read().decode('utf-8')
    )
    return resp['response'][0]
user_data = get_user_by_name('va.sinikov')
print(user_data)

{'id': 10530059, 'first_name': 'Вася', 'last_name': 'Сиников', 'is_closed': False, 'can_access_closed': True, 'photo_50': 'https://pp.userapi.com/c630716/v630716059/49338/-iB3XGuYES8.jpg?ava=1'}


In [14]:
def get_friends(uid):
    _friends = json.loads(urllib.request.urlopen(
        'https://api.vk.com/method/friends.get?v='+vk_api_version+'&user_id='+str(uid)
        +'&fields=name,photo_50&access_token='+token).read().decode('utf-8'))
    return _friends['response']['items']

In [15]:
import re
import tqdm
import time
UID_RE = 'vk\.com/([a-z0-9\.\_]+)'

uid = links.dropna().sample(1).iloc[0]
print(uid)
unames = re.findall(UID_RE, uid)
print(unames)
if unames:
    user_data = get_user_by_name(unames[0])
    print(user_data)
    friends = get_friends(user_data['id'])
    print(len(friends))

https://vk.com/galgashova
['galgashova']
{'id': 5667261, 'first_name': 'Стася', 'last_name': 'Галгашова', 'is_closed': False, 'can_access_closed': True, 'photo_50': 'https://pp.userapi.com/c841629/v841629756/33750/pigS8NyGXeY.jpg?ava=1'}
795


In [16]:
user_data

{'can_access_closed': True,
 'first_name': 'Стася',
 'id': 5667261,
 'is_closed': False,
 'last_name': 'Галгашова',
 'photo_50': 'https://pp.userapi.com/c841629/v841629756/33750/pigS8NyGXeY.jpg?ava=1'}

In [ ]:
unames

['galgashova']

In [ ]:
members = {}
member_friends = {}
for row_id in tqdm.tqdm_notebook(np.arange(excel.shape[0])):
    row = excel.iloc[row_id].fillna('')
    vk_url = row[URL_COL]
    unames = re.findall(UID_RE, vk_url)
    if len(unames) == 0 or unames[0] == 'feed': 
        print('"{}" is invalid link'.format(vk_url))
        continue
    user_data = get_user_by_name(unames[0])
    if user_data['is_closed'] and not user_data['can_access_closed']:
        print('"{}" is closed'.format(unames[0]))
        continue
    members[row_id] = user_data
    friends = get_friends(user_data['id'])
    member_friends[user_data['id']] = friends
    print('{:30} -> {}'.format(vk_url, len(friends)))
    time.sleep(0.3)

https://vk.com/timuraz         -> 454
"" is invalid link
https://vk.com/id119458406     -> 813
https://vk.com/hello_dashka    -> 568
https://m.vk.com/maxmold       -> 1271
https://vk.com/ksela           -> 276
https://vk.com/ikolankov       -> 1143
https://vk.com/id166330633     -> 250
https://vk.com/revolkov        -> 1403
https://vk.com/vladislav_rasskazov -> 273
https://vk.com/yulia.gorodishcher -> 646
https://vk.com/svoeobuchnaya   -> 425
https://vk.com/elenasachkova   -> 169
https://vk.com/vika.lipatova   -> 277
"" is invalid link
"" is invalid link
https://m.vk.com/id4368063     -> 144
"https://vk.com/feed" is invalid link
https://vk.com/heysoch         -> 1271
https://vk.com/da_ilienkov     -> 818
https://vk.com/az_zakria       -> 482
"id31870485" is closed
https://vk.com/mvishnevskii    -> 256
https://vk.com/a.lantsoff      -> 1086
https://m.vk.com/id7263977     -> 215
https://vk.com/nickolaevegor   -> 170
https://vk.com/alievaleila     -> 229
https://vk.com/certainways     -> 

In [ ]:
print(len(members))

In [ ]:
uid2rowid = {v['id']: k for k, v in members.items()}
rowid2name = {k: '{} {}'.format(excel.iloc[k, 1], excel.iloc[k, 2]) for k, v in members.items()}

In [ ]:
from collections import defaultdict

In [ ]:
member_names = [rowid2name[rowid] for rowid in members.keys()]
membername2rowid = {rowid2name[rowid]: rowid for rowid in members.keys()}
membername2friendnames = defaultdict(list)
pairs = []
for uid, friends in member_friends.items():
    left_name = rowid2name[uid2rowid[uid]]
    for friend in friends:
        if friend['id'] in uid2rowid:
            right_name = rowid2name[uid2rowid[friend['id']]]
            pairs.append([left_name, right_name])
            membername2friendnames[left_name].append(right_name)
len(pairs)

In [ ]:
membername2idx = {name: idx for idx, name in enumerate(member_names)}

In [ ]:
pairs_df = pd.DataFrame(pairs, columns=['first', 'second'])
pairs_df.head()

In [ ]:
matrix = pairs_df.pivot_table(index='first', columns='second', aggfunc=len).fillna(0).astype(int)
matrix.shape

In [ ]:
# matrix.to_excel('matrix.xlsx')

# Визуализация

Теперь подключим библиотеку networkx, которая позволяет проводить расчёты на графах и визуализировать их.

In [ ]:
import networkx as nx

Создадим граф, добавив туда только друзей иосновного пользователя и связи между ними.

In [ ]:
G = nx.Graph()
G.add_nodes_from(member_names)
G.add_edges_from(pairs)

Каков размер графа?

In [ ]:
print(G.number_of_nodes(), G.number_of_edges())

Воспользуемся готовым алгоритмом визуализации графа. Видим, что есть много точек, ни с кем не связанных.

In [ ]:
G.is_directed()

In [ ]:
pd.Series(dict(G.degree)).sort_values(ascending=False)

In [ ]:
nx.draw(G, node_size=1)

Оказывается, библиотека networkx умеет отвечать на вопросы о свойствах графа, например, о том, является ли он связным.

In [ ]:
nx.is_connected(G)

Посчитаем число соединённых компонент в графе.

In [ ]:
subgraphs = [sg for sg in nx.connected_component_subgraphs(G)]
print(len(subgraphs))

In [ ]:
comp_lengths = [len(sg.nodes()) for sg in subgraphs]
print(comp_lengths)

In [ ]:
G1 = subgraphs[np.argmax(comp_lengths)]

In [ ]:
nx.degree(G1)

In [ ]:
degrees=pd.Series(dict(nx.degree(G1)))
plt.hist(degrees, bins=10);

Тот алгоритм рисования графа, которым мы воспользовались, основан на воображаемых "пружинках" между узлами.

In [ ]:
plt.figure(figsize = (15,10))
labels = {f:f for f in member_names if f in G1.nodes()}
nx.draw_spring(
    G1, node_size=1, width=0.3, labels = labels, font_size=10, font_family='Verdana', 
    k=10, iterations=500, threshold=1e-10
);

Есть и другие методы визуализации - например, с помощью концентрических окружностей.

In [ ]:
plt.figure(figsize = (10,10))
nx.draw_shell(G1, node_size = 1, width  = 0.1, labels = labels)

Если у вас очень много друзей, можно нарисовать случайный подграф вашего графа

In [ ]:
random_friends = np.random.choice(range(len(G1)), size=30, replace = False)
all_nodes = G1.nodes()
G2 = G1.subgraph([member_names[i] for i in random_friends])
subgraphs2 = [sg for sg in nx.connected_component_subgraphs(G2)]
G2 = subgraphs2[np.argmax([len(sg) for sg in subgraphs2])]
labels2 = {f:f for f in member_names if f in G2.nodes()}

plt.figure(figsize = (10,8))
nx.draw_spring(G2, node_size = 5, width  = 0.3, labels = labels2, font_size = 10, font_family  = 'Verdana')

# Экспорт данных

In [ ]:


coord = nx.kamada_kawai_layout(G1, )
xy = np.stack(list(coord.values()))
plt.plot(xy[:,0], xy[:, 1])

In [ ]:
def get_bachelor(row):
    if row.loc[BACHELOR_COL] is np.nan:
        return row.loc[OTHER_BACH_COL]
    return row.loc[BACHELOR_COL]

In [ ]:
sub = list(G1.nodes)

export = {
    "members": [
        {
            "name": member_name, 
            "img": excel.loc[membername2rowid[member_name], PHOTO_COL],
            "url": excel.loc[membername2rowid[member_name], URL_COL], 
            "x": coord[member_name][0] * 0.5 + 0.5,
            "y": coord[member_name][1] * 0.5 + 0.5,
            "friends": [c for c in membername2friendnames[member_name] if c in sub],
            "vk_img": members[membername2rowid[member_name]]['photo_50'],
            'city': excel.loc[membername2rowid[member_name], CITY_COL],
            'bachelor': get_bachelor(excel.loc[membername2rowid[member_name]]),
            'job': excel.loc[membername2rowid[member_name], JOB_COL],
            'interests': excel.loc[membername2rowid[member_name], INTERESTS_COL],
            'hobby': excel.loc[membername2rowid[member_name], HOBBY_COL],
            'fun': excel.loc[membername2rowid[member_name], FUN_COL],
            'inflections': get_inflections(excel.loc[membername2rowid[member_name], NAME_COL]),
            'in_club': bool(excel.loc[membername2rowid[member_name], IN_CLUB_COL]),
            'presence': bool(excel.loc[membername2rowid[member_name], PRESENCE_COL]),
            'idx': i
        }
        for i, member_name in enumerate(sub)
    ]
}
export

In [ ]:
import codecs
with codecs.open('kv_soc_graph/graph_data.json', 'w', encoding='utf-8') as f:
    f.write('graph_data=' + json.dumps(export, ensure_ascii=False, indent=2))